In [52]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime , timedelta
from chromedriver_py import binary_path
from sqlalchemy import create_engine
from selenium import webdriver
from bs4 import BeautifulSoup
from pangres import upsert
import pandas as pd
import requests
import time
import bs4
import sys
import re 
import os

In [ ]:
download_path = os.path.join(os.path.expanduser('~'), 'Downloads')
file = 'Example.xlsx'
os.path.join(download_path,file)

In [70]:
download_path = os.path.join(os.path.expanduser('~'), 'Downloads')
download_path = 'D:\\Tutorial\\Git\\MeltMultiCond\\'
file = 'Example.xlsx'

data = pd.read_excel(os.path.join(download_path,file),header=None)
data.columns = data.loc[3]
report_date = datetime.strptime(data['Article'].loc[0].replace('Sales Period: ','').split(' - ')[1],'%b-%d-%Y').strftime('%Y-%m-%d')
data = data.drop(['Description'], axis=1)
data.columns = range(len(data.columns))
data

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Sales Period: Mar-01-2024 - Mar-17-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN
2,NaN,Branch_Code:1001,NaN,Branch_Code:1002,NaN,Branch_Code:1004,NaN,Branch_Code:1015,NaN,Branch_Code:1024,NaN,Total,NaN
3,Article,Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB)
4,154312001,5,300,4,240,1,60,NaN,NaN,NaN,NaN,10,600
5,154312002,1,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,40
6,NaN,6,340,4,240,1,60,NaN,NaN,NaN,NaN,11,640
7,154312051,1,100,NaN,NaN,NaN,NaN,4,400,NaN,NaN,5,500
8,154312052,9,1800,3,600,1,200,1,200,7,1400,21,4200
9,NaN,10,1900,3,600,5,200,5,600,7,1400,26,4700


In [71]:
#Create Column Name
data_t = data.loc[2:3].transpose()
data_t.columns = ['column1','column2']
data_t['column1'] = data_t['column1'].fillna(method='ffill', axis=0)
data_t['column3'] = data_t.apply(lambda row: f"{str(row['column1'][-4:])} {str(row['column2'][6:]).replace('(','').replace('.','').replace(')','')}" if pd.notna(row['column1']) and pd.notna(row['column2']) else 'Article', axis=1)
data.columns = data_t['column3'].to_list()
data = data[data['Article'].apply(lambda d: isinstance(d,int))]
data


,Article,1001 Qty,1001 THB,1002 Qty,1002 THB,1004 Qty,1004 THB,1015 Qty,1015 THB,1024 Qty,1024 THB,al Qty,al THB
4,154312001,5,300,4,240,1,60,NaN,NaN,NaN,NaN,10,600
5,154312002,1,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,40
7,154312051,1,100,NaN,NaN,NaN,NaN,4,400,NaN,NaN,5,500
8,154312052,9,1800,3,600,1,200,1,200,7,1400,21,4200


In [ ]:
#Melt
data_melt = pd.melt(data, id_vars=['Article'],var_name='type',value_name='value')
data_melt = data_melt[(data_melt['value'].notna()) & (data_melt['type'].apply(lambda d: 'al' not in d))]
data_melt = data_melt.sort_values(['Article','type']).reset_index(drop=True)
data_qty = data_melt[data_melt['type'].apply(lambda d: 'Qty' in d)]
data_amount = data_melt[data_melt['type'].apply(lambda d: 'THB' in d)]

data_qty['branch_id'] = data_qty['type'].apply(lambda d: d[:4])
data_qty.columns = ['shop_barcode','drop','qty','branch_id']
data_qty = data_qty[['shop_barcode','branch_id','qty']]

data_amount['branch_id'] = data_amount['type'].apply(lambda d: d[:4])
data_amount.columns = ['shop_barcode','drop','amount_exvat','branch_id']
data_amount = data_amount[['shop_barcode','branch_id','amount_exvat']]

data = data_qty.merge(data_amount,how='inner',on=['shop_barcode','branch_id'])
data.columns = ['shop_barcode', 'branch_id', 'qty', 'amount_exvat']
data['sale_date'] = report_date

In [73]:
data

,shop_barcode,branch_id,qty,amount_exvat,sale_date
0,154312001,1001,5,300,2024-03-17
1,154312001,1002,4,240,2024-03-17
2,154312001,1004,1,60,2024-03-17
3,154312002,1001,1,40,2024-03-17
4,154312051,1001,1,100,2024-03-17
5,154312051,1015,4,400,2024-03-17
6,154312052,1001,9,1800,2024-03-17
7,154312052,1002,3,600,2024-03-17
8,154312052,1004,1,200,2024-03-17
9,154312052,1015,1,200,2024-03-17
